In [34]:
# coding: utf-8
from pathlib import Path

import pandas as pd
from sklearn.metrics import mean_squared_error
import sklearn.model_selection
import lightgbm as lgb


In [35]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

In [36]:
data = pd.read_csv('../Data/countyVLivEdu.csv')
data

,Unnamed: 0,case_id,state_x,isMetro,areaname,county,family_member_count,housing_cost,food_cost,transportation_cost,...,Col,state_y,county_name,totalvotes,DEMOCRAT,GREEN,LIBERTARIAN,OTHER,REPUBLICAN,Abv
0,0,1,AL,True,"Montgomery, AL MSA",Autauga,1p0c,8505.72876,3454.91712,10829.16876,...,28.131469,ALABAMA,Autauga,27770,0.270184,0.0,0.000000,0.015448,0.714368,AL
1,1,1,AL,True,"Montgomery, AL MSA",Autauga,1p1c,12067.50240,5091.70788,11588.19288,...,28.131469,ALABAMA,Autauga,27770,0.270184,0.0,0.000000,0.015448,0.714368,AL
2,2,1,AL,True,"Montgomery, AL MSA",Autauga,1p2c,12067.50240,7460.20308,12361.77720,...,28.131469,ALABAMA,Autauga,27770,0.270184,0.0,0.000000,0.015448,0.714368,AL
3,3,1,AL,True,"Montgomery, AL MSA",Autauga,1p3c,15257.15040,9952.23924,13452.18600,...,28.131469,ALABAMA,Autauga,27770,0.270184,0.0,0.000000,0.015448,0.714368,AL
4,4,1,AL,True,"Montgomery, AL MSA",Autauga,1p4c,15257.15040,12182.21400,13744.59840,...,28.131469,ALABAMA,Autauga,27770,0.270184,0.0,0.000000,0.015448,0.714368,AL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22185,22185,3171,WY,False,"Weston County, WY",Weston,2p0c,8316.00000,6010.65672,14790.36240,...,18.397383,WYOMING,Weston,3560,0.101124,0.0,0.012921,0.013202,0.872753,WY
22186,22186,3171,WY,False,"Weston County, WY",Weston,2p1c,10956.00000,7480.86696,15611.58600,...,18.397383,WYOMING,Weston,3560,0.101124,0.0,0.012921,0.013202,0.872753,WY
22187,22187,3171,WY,False,"Weston County, WY",Weston,2p2c,10956.00000,9474.35748,16163.90280,...,18.397383,WYOMING,Weston,3560,0.101124,0.0,0.012921,0.013202,0.872753,WY
22188,22188,3171,WY,False,"Weston County, WY",Weston,2p3c,13632.00000,11567.46972,17118.38400,...,18.397383,WYOMING,Weston,3560,0.101124,0.0,0.012921,0.013202,0.872753,WY


In [37]:
data[['total_cost', 'GREEN', 'REPUBLICAN', 'DEMOCRAT', 'LIBERTARIAN', 'OTHER', 
            'noHS', 'HS',	'someCol',	'Col']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22190 entries, 0 to 22189
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   total_cost   22190 non-null  float64
 1   GREEN        22190 non-null  float64
 2   REPUBLICAN   22190 non-null  float64
 3   DEMOCRAT     22190 non-null  float64
 4   LIBERTARIAN  22190 non-null  float64
 5   OTHER        22190 non-null  float64
 6   noHS         22190 non-null  float64
 7   HS           22190 non-null  float64
 8   someCol      22190 non-null  float64
 9   Col          22190 non-null  float64
dtypes: float64(10)
memory usage: 1.7 MB


In [42]:


# y = 1 * x_0 + 2 * x_1 + 3
X = data[['median_family_income','GREEN', 'REPUBLICAN', 'DEMOCRAT', 'LIBERTARIAN', 'OTHER', 
            'noHS', 'HS',	'someCol',	'Col']]
y = data['total_cost']
reg = LinearRegression().fit(X, y)
reg.score(X, y)


0.13069189559231642

In [40]:
# load or create your dataset
regression_example_dir = Path('../Data/countyVLivEdu.csv').absolute().parents[1] / 'regression'
df_train = pd.read_csv(str(regression_example_dir / 'regression.train'), header=None, sep='\t')
df_test = pd.read_csv(str(regression_example_dir / 'regression.test'), header=None, sep='\t')


FileNotFoundError: [Errno 2] No such file or directory: 'c:\\School\\Stat386\\Stat-386-Project\\modules\\..\\regression\\regression.train'

In [ ]:


y_train = df_train[0]
y_test = df_test[0]
X_train = df_train.drop(0, axis=1)
X_test = df_test.drop(0, axis=1)

# create dataset for lightgbm
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

# specify your configurations as a dict
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2', 'l1'},
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

print('Starting training...')
# train
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=20,
                valid_sets=lgb_eval,
                callbacks=[lgb.early_stopping(stopping_rounds=5)])

print('Saving model...')
# save model to file
gbm.save_model('model.txt')

print('Starting predicting...')
# predict
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)
# eval
rmse_test = mean_squared_error(y_test, y_pred) ** 0.5
print(f'The RMSE of prediction is: {rmse_test}')

NameError: name 'df_train' is not defined